# 1.1.3.1: Producto Punto (Dot Product)

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Calcular** el producto punto de dos vectores usando su definición algebraica y su implementación en NumPy.
- **Interpretar** el signo (positivo, negativo, cero) del producto punto en términos del ángulo entre los vectores.
- **Utilizar** el producto punto para determinar si dos vectores de datos son **ortogonales** (sin relación lineal).
- **Aplicar** el producto punto para calcular el **ángulo** y la **proyección** de un vector sobre otro.
- **Medir la similitud** entre vectores de datos (e.g., perfiles de clientes o documentos) usando la **Similitud del Coseno**.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def plot_vectors(vectors, colors=None, labels=None, title='Vectores', ax=None):
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(7, 7))
    
    if colors is None:
        colors = ['#0072B2', '#E69F00', '#D55E00', '#CC79A7', '#56B4E9']
        
    for i, v in enumerate(vectors):
        ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1, 
                  color=colors[i % len(colors)], label=labels[i] if labels and i < len(labels) else f'v{i+1}')

    all_points = np.array(vectors)
    max_val = np.max(np.abs(all_points)) * 1.2
    if max_val == 0: max_val=1
    ax.set_xlim([-max_val, max_val])
    ax.set_ylim([-max_val, max_val])
    ax.set_aspect('equal')
    ax.grid(True, linestyle='--')
    ax.axhline(0, color='black', lw=0.5)
    ax.axvline(0, color='black', lw=0.5)
    ax.set_title(title)
    ax.legend()
    if standalone:
        plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

El producto punto es una herramienta para medir relaciones entre vectores. Por lo tanto, usaremos nuestros generadores para crear pares o grupos de vectores con relaciones interesantes que podamos descubrir y cuantificar.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_business_data import create_business_data
from src.data_generation.create_special_matrices import create_special_matrices

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets y Vectores para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Perfiles de Estudiantes)
# Usaremos los vectores de estudiantes para medir similitud. ¿Dos estudiantes tienen un perfil
# de [horas, calificación] similar, incluso si sus valores absolutos son diferentes?
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=50)

# 💡 CONTEXTO PEDAGÓGICO: Perfiles de Negocio para Similitud
# Un caso de uso clásico es encontrar productos similares. Generaremos un dataset
# de negocio para practicar el cálculo de la similitud del coseno entre perfiles de productos.
datos_negocio = create_business_data(rng, n_samples=20)

# 💡 CONTEXTO PEDAGÓGICO: Vectores para Proyecciones y Ángulos
# Para los ejemplos geométricos puros, definiremos vectores explícitamente para tener control total.
vec_a = np.array([4, 1])
vec_b = np.array([2, 3])
vec_ortogonal_a_b = np.array([-3, 2])

print("Datasets y vectores generados y listos para usar.")

## 1. Definición y Significado del Producto Punto

El producto punto (o producto escalar) es una operación entre dos vectores de la misma dimensión que resulta en un **único número (un escalar)**. Tiene dos definiciones equivalentes que nos dan el "cómo" computacional y el "qué significa" geométrico.

### Definición Algebraica (El "Cómo")

Se calcula multiplicando los componentes correspondientes de los vectores y sumando los resultados.
$$ \vec{v} \cdot \vec{w} = v_1w_1 + v_2w_2 + \dots + v_nw_n = \sum_{i=1}^{n} v_i w_i = \vec{v}^T \vec{w}$$

> **Aplicación en Data Science:** Esta operación es el núcleo de la mayoría de los modelos lineales y redes neuronales. La predicción de un modelo a menudo es una suma ponderada, que es exactamente un producto punto: `predicción = vector_de_features · vector_de_pesos`.

### Interpretación Geométrica (El "Qué Significa")

La segunda definición lo conecta directamente con la geometría, revelando su verdadero propósito:
$$ \vec{v} \cdot \vec{w} = \|\vec{v}\| \|\vec{w}\| \cos(\theta) $$
donde $\theta$ es el ángulo entre los vectores.

**La Gran Intuición:** El producto punto mide el **grado de alineación** entre dos vectores, escalado por sus magnitudes.
- **$\vec{v} \cdot \vec{w} > 0$**: Ángulo agudo ($< 90^\circ$). Apuntan en direcciones generales similares.
- **$\vec{v} \cdot \vec{w} = 0$**: Ángulo recto ($= 90^\circ$). Son **ortogonales** (perpendiculares).
- **$\vec{v} \cdot \vec{w} < 0$**: Ángulo obtuso ($> 90^\circ$). Apuntan en direcciones opuestas.

### Ejemplo Demostrativo 1: Cálculo e Interpretación del Signo

In [ ]:
# 1. DATOS: Un vector de referencia 'v' y otros tres vectores con diferentes alineaciones.
v = np.array([3, 1])
w_agudo = np.array([2, 2])    # Apunta en una dirección similar
w_ortogonal = np.array([-1, 3]) # Perpendicular
w_obtuso = np.array([-2, 1])   # Apunta en una dirección opuesta

# 2. APLICACIÓN: Calculamos el producto punto con el operador @ de NumPy
dot_agudo = v @ w_agudo
dot_ortogonal = v @ w_ortogonal
dot_obtuso = v @ w_obtuso

# 3. INTERPRETACIÓN
print(f"v · w_agudo     = {dot_agudo} (> 0, el ángulo es agudo)")
print(f"v · w_ortogonal = {dot_ortogonal} (= 0, el ángulo es recto -> Ortogonales)")
print(f"v · w_obtuso    = {dot_obtuso} (< 0, el ángulo es obtuso)")

# 4. VISUALIZACIÓN
plot_vectors([v, w_agudo, w_ortogonal, w_obtuso], 
             labels=['v', 'w_agudo', 'w_ortogonal', 'w_obtuso'],
             title='Interpretando el Signo del Producto Punto')

## 2. Aplicaciones Clave: Ángulos, Proyecciones y Similitud

### A. Similitud del Coseno
Si reordenamos la fórmula geométrica, obtenemos una medida pura de alineación, independiente de las magnitudes de los vectores. Esta es la **similitud del coseno**, una métrica fundamental en Procesamiento del Lenguaje Natural (NLP) y Sistemas de Recomendación.
$$ \text{sim}(\vec{v}, \vec{w}) = \cos(\theta) = \frac{\vec{v} \cdot \vec{w}}{\|\vec{v}\| \|\vec{w}\|} $$
Su valor va de -1 (perfectamente opuestos) a 1 (perfectamente alineados), con 0 significando ortogonales (sin similitud).

### Ejemplo Demostrativo 2: Similitud entre Perfiles de Productos

In [ ]:
# 1. DATOS: Perfiles de 3 productos de nuestro dataset de negocio. 
# Cada perfil es un vector [precio, gasto_marketing, calificacion_cliente]
perfiles = datos_negocio[['precio', 'gasto_marketing', 'calificacion_cliente']].values
prod_A = perfiles[0]
prod_B = perfiles[1]
prod_C = perfiles[2]

# 2. APLICACIÓN: Calculamos la similitud del coseno entre A y los otros.
def cosine_similarity(v, w):
    dot_product = np.dot(v, w)
    norm_v = np.linalg.norm(v)
    norm_w = np.linalg.norm(w)
    return dot_product / (norm_v * norm_w)

sim_AB = cosine_similarity(prod_A, prod_B)
sim_AC = cosine_similarity(prod_A, prod_C)

# 3. INTERPRETACIÓN
print(f"Perfil Producto A: {np.round(prod_A, 2)}")
print(f"Perfil Producto B: {np.round(prod_B, 2)}")
print(f"Perfil Producto C: {np.round(prod_C, 2)}")
print(f"\nSimilitud(A, B) = {sim_AB:.3f}")
print(f"Similitud(A, C) = {sim_AC:.3f}")
print("\nConclusión: El producto A es mucho más similar en 'perfil' al producto B que al C.")

### B. Proyección Vectorial
La proyección de $\vec{v}$ sobre $\vec{w}$ (denotada $\text{proj}_{\vec{w}}(\vec{v})$) es la "sombra" que $\vec{v}$ proyecta sobre la línea definida por $\vec{w}$. Se calcula en dos pasos:

1.  **Calcular la longitud de la sombra (componente escalar):** $c = \frac{\vec{v} \cdot \vec{w}}{\|\vec{w}\|^2}$
2.  **Convertir esa longitud en un vector:** $\text{proj}_{\vec{w}}(\vec{v}) = c \cdot \vec{w}$

> **Aplicación en Data Science:** La Proyección es la base de la **Regresión por Mínimos Cuadrados (OLS)**. El vector de predicciones $\hat{y}$ es la proyección del vector de valores reales $y$ sobre el espacio generado por las columnas (features) de la matriz de datos $X$.

### Ejemplo Demostrativo 3: Proyección Geométrica Pura

In [ ]:
# 1. DATOS: Vectores definidos para la demostración
v = np.array([2, 3])
w = np.array([4, 1])

# 2. APLICACIÓN: Calculamos la proyección de v sobre w.
componente_escalar = (v @ w) / (w @ w) # Usamos v@v = ||v||^2
proyeccion_vectorial = componente_escalar * w

# 3. INTERPRETACIÓN Y VISUALIZACIÓN
print(f"Vector v = {v}")
print(f"Vector w = {w}")
print(f"La 'sombra' de v sobre w tiene una longitud relativa de {componente_escalar:.2f} veces w.")
print(f"El vector de proyección es {np.round(proyeccion_vectorial, 2)}.")

fig, ax = plt.subplots(figsize=(8, 8))
ax.plot([v[0], proyeccion_vectorial[0]], [v[1], proyeccion_vectorial[1]], 'r--', label='Componente Ortogonal')
plot_vectors([v, w, proyeccion_vectorial], 
             labels=['v', 'w', 'Proyección de v sobre w'], 
             title='Visualización de Proyección Vectorial',
             ax=ax)

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Cálculo Básico e Interpretación del Signo ===

In [ ]:
# DATOS
a = np.array([4, -2])
b = np.array([1, 2])

# TODO 1: Calcula el producto punto de a y b.
# PISTA: Usa el operador '@' de NumPy.
producto_punto = # COMPLETAR

# TODO 2: Basado en el signo del resultado, ¿qué tipo de ángulo hay entre a y b?
interpretacion = "" # Escribe 'agudo', 'recto' u 'obtuso'
# COMPLETAR

# VERIFICACIÓN
assert producto_punto == 0, "El cálculo es incorrecto."
assert interpretacion == "recto", "La interpretación es incorrecta."
print("✅ ¡Correcto! El producto punto es 0, lo que significa que los vectores son ortogonales (ángulo recto).")
plot_vectors([a, b], labels=['a', 'b'])

### === EJERCICIO GUIADO 2: Encontrar un Vector Ortogonal ===


In [ ]:
# DATOS
v = np.array([5, 2])

# Queremos encontrar un vector w = [x, y] tal que v · w = 0.
# Esto nos da la ecuación 5x + 2y = 0.

# TODO 1: Encuentra un par de valores (x, y) no nulos que satisfagan la ecuación.
# PISTA: Un truco fácil en 2D es intercambiar los componentes y cambiar el signo de uno: [y, -x] o [-y, x].
w = # COMPLETAR

# VERIFICACIÓN
assert v @ w == 0, "El producto punto no es cero. Los vectores no son ortogonales."
print(f"✅ ¡Correcto! El vector {w} es ortogonal a {v}.")
plot_vectors([v, w], labels=['v', 'w'])

### === EJERCICIO GUIADO 3: Calcular el Ángulo entre Vectores ===


In [ ]:
# DATOS
v_estudiante_A = datos_estudiantes.iloc[5].values # Vector del estudiante A
v_estudiante_B = datos_estudiantes.iloc[10].values # Vector del estudiante B

# TODO 1: Calcula el producto punto de los dos vectores.
dot_product = # COMPLETAR

# TODO 2: Calcula la norma (magnitud) de cada vector.
norm_A = # COMPLETAR
norm_B = # COMPLETAR

# TODO 3: Calcula el coseno del ángulo usando la fórmula.
cos_theta = # COMPLETAR

# TODO 4: Calcula el ángulo en grados.
# PISTA: Usa np.arccos() y np.rad2deg().
angulo_grados = # COMPLETAR

# VERIFICACIÓN
assert 0 <= cos_theta <= 1, "El coseno debería estar entre 0 y 1 para estos datos."
print(f"✅ ¡Cálculo correcto! El ángulo entre los perfiles de los estudiantes es de {angulo_grados:.2f} grados.")

### === EJERCICIO GUIADO 4: Calcular Similitud del Coseno ===

In [ ]:
# DATOS: Los mismos vectores de estudiante del ejercicio anterior.
v_estudiante_A = datos_estudiantes.iloc[5].values
v_estudiante_B = datos_estudiantes.iloc[10].values

# La similitud del coseno es simplemente el coseno del ángulo que ya calculaste.

# TODO: Calcula la similitud del coseno usando la fórmula completa en una línea.
similitud = # COMPLETAR

# VERIFICACIÓN
cos_theta_manual = (v_estudiante_A @ v_estudiante_B) / (np.linalg.norm(v_estudiante_A) * np.linalg.norm(v_estudiante_B))
assert np.isclose(similitud, cos_theta_manual)
print(f"✅ ¡Correcto! La similitud del coseno entre los estudiantes es {similitud:.3f}.")
print("Interpretación: Un valor cercano a 1 indica que los perfiles son muy similares.")

### === EJERCICIO GUIADO 5: Calcular la Proyección Vectorial ===

In [ ]:
# DATOS
v_datos = np.array([3, 5])
v_eje = np.array([1, 1]) # El eje sobre el cual proyectamos (línea y=x)

# TODO 1: Calcula el componente escalar (la longitud de la sombra).
# PISTA: (v_datos · v_eje) / (v_eje · v_eje)
comp_escalar = # COMPLETAR

# TODO 2: Calcula el vector de proyección multiplicando el componente escalar por el vector del eje.
proyeccion = # COMPLETAR

# VERIFICACIÓN
assert np.isclose(comp_escalar, 4.0)
assert np.allclose(proyeccion, np.array([4, 4]))
print("✅ ¡Proyección calculada correctamente!")
plot_vectors([v_datos, v_eje, proyeccion], labels=['Datos', 'Eje', 'Proyección'])

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Cálculo e Interpretación

**A1 (🟢 Fácil):** Calcula el producto punto de $\vec{u}=[3, -1, 4]$ y $\vec{v}=[2, 5, -1]$.

**A2 (🟢 Fácil):** Sin calcular el ángulo, determina si el ángulo entre $\vec{a}=[5,2]$ y $\vec{b}=[-1,3]$ es agudo, recto u obtuso.

**A3 (🟢 Fácil):** Haz lo mismo para $\vec{c}=[-2, -3]$ y $\vec{d}=[1, -5]$.

**A4 (🟡 Medio):** ¿Qué resultado da el producto punto de un vector consigo mismo, $\vec{v} \cdot \vec{v}$? ¿Con qué otra medida que ya conoces se relaciona?

**A5 (🟡 Medio):** Si $\vec{u} \cdot \vec{v} = -15$, $\|\vec{u}\|=5$ y $\|\vec{v}\|=4$, ¿cuál es el coseno del ángulo entre ellos?

**A6 (🔴 Reto):** Demuestra la propiedad distributiva: $\vec{u} \cdot (\vec{v} + \vec{w}) = (\vec{u} \cdot \vec{v}) + (\vec{u} \cdot \vec{w})$ usando tres vectores 3D aleatorios.

### Parte B: Ortogonalidad

**B1 (🟢 Fácil):** Verifica si los vectores $\vec{a}=[6, -2]$ y $\vec{b}=[1, 3]$ son ortogonales.

**B2 (🟢 Fácil):** ¿Son ortogonales los vectores de la base canónica $\vec{i}=[1,0,0]$ y $\vec{j}=[0,1,0]$?

**B3 (🟡 Medio):** Encuentra un valor para $k$ tal que el vector $\vec{u}=[2, k, 1]$ sea ortogonal a $\vec{v}=[4, 1, -3]$.

**B4 (🟡 Medio):** Encuentra un vector 2D que sea ortogonal al vector del primer estudiante en `datos_estudiantes`.

**B5 (🔴 Reto):** Encuentra un vector $\vec{w}$ que sea ortogonal a la vez a $\vec{u}=[1,1,1]$ y a $\vec{v}=[1,2,3]$. (Pista: esto te dará un sistema de dos ecuaciones con tres incógnitas).

### Parte C: Ángulos y Similitud del Coseno

**C1 (🟢 Fácil):** Calcula el ángulo (en grados) entre $\vec{a}=[1,1]$ y $\vec{b}=[0, 2]$.

**C2 (🟡 Medio):** Calcula el ángulo entre los vectores $\vec{u}=[1, 2, 3]$ y $\vec{v}=[4, 5, 6]$.

**C3 (🟡 Medio):** Usando `datos_negocio`, toma los perfiles de los productos en los índices 3 y 4. Calcula la similitud del coseno entre ellos.

**C4 (🔴 Reto):** Encuentra al estudiante en `datos_estudiantes` cuyo perfil es más similar al perfil del estudiante en el índice 0, usando la similitud del coseno. (Pista: itera sobre todos los demás estudiantes y quédate con el que dé la similitud más alta).

### Parte D: Proyecciones

**D1 (🟢 Fácil):** Calcula la proyección de $\vec{v}=[5, 1]$ sobre $\vec{w}=[3, 0]$ (el eje x).

**D2 (🟡 Medio):** Calcula la proyección de $\vec{v}=[-1, 3]$ sobre $\vec{w}=[2, 2]$ y visualiza los tres vectores.

**D3 (🟡 Medio):** Define un "vector de mercado" $\vec{m}=[100, 50]$ para `datos_negocio`. Proyecta el perfil del producto en el índice 8 sobre este vector de mercado.

**D4 (🔴 Reto):** El vector $\vec{v}_{\perp} = \vec{v} - \text{proj}_{\vec{w}}(\vec{v})$ es el componente de $\vec{v}$ que es ortogonal a $\vec{w}$. Calcula este vector para $\vec{v}=[2,3]$ y $\vec{w}=[4,1]$ y verifica que $\vec{v}_{\perp} \cdot \vec{w}$ es (casi) cero.

---

## ✅ Mini-Quiz de Autoevaluación

1. ¿Cuál es el tipo de dato del resultado de un producto punto? (Vector, escalar, matriz, etc.)
2. ¿Cuál es el valor del producto punto de dos vectores no nulos si y solo si son ortogonales?
3. Si la similitud del coseno entre dos vectores es -1, ¿qué puedes decir de ellos?
4. ¿Es el producto punto una operación conmutativa? Es decir, ¿es $\vec{u} \cdot \vec{v}$ siempre igual a $\vec{v} \cdot \vec{u}$?
5. La proyección de $\vec{v}$ sobre $\vec{w}$, ¿tiene la misma dirección que $\vec{v}$ o que $\vec{w}$?

## 🚀 Próximos Pasos

¡Felicidades! Has dominado una de las operaciones más versátiles y fundamentales del álgebra lineal.

- En el notebook **`1.1.3.2_Determinante_y_Eigenvalores`**, exploraremos propiedades intrínsecas de las matrices cuadradas. El determinante nos dirá cómo una transformación escala áreas/volúmenes, y los eigenvalores/eigenvectores nos revelarán los ejes de transformación preferidos de una matriz, la base de algoritmos como PCA.